# Computation Theory

In [ ]:
#imports

import numpy as np

### Problem 1: Binary Words and Operations

In [ ]:
def Parity(x,y,z):
    """
    function takes in three parameters x,y,z (integers).
    Converts them to 32-bit unsigned integers
    returns the bitwise XOR of the three values.

    Args:
        x (int): first integer input
        y (int): second integer input
        z (int): third integer input

    Returns:
        int: bitwise XOR of x, y, and z as a 32-bit unsigned integer

    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32(x ^ y ^ z)


def Choose(x,y,z):
    """
    function takes in three parameters x,y,z (integers).
    Converts them to 32-bit unsigned integers.
    Chooses bits from y and z based on the bits of x.
    If a bit in x is 1, the corresponding bit from y is chosen.
    If a bit in x is 0, the corresponding bit from z is chosen.

    Args:
        x (int): first integer input
        y (int): second integer input
        z (int): third integer input

    Returns:
        32-bit unsigned integer: result of the expression (x & y) ^ (~x & y)

    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ (~x & y))

def Majority(x,y,z):
    """
    function takes in three parameters x,y,z (integers).
    Converts them to 32-bit unsigned integers.
    Returns the majority value among the three inputs.

    Args:
        x (int): first integer input
        y (int): second integer input
        z (int): third integer input

    Returns:
        32-bit unsigned integer: result of the expression (x & y) ^ (x & z) ^ (y & z) 
        
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ (x & z) ^ (y & z))



### Problem 2: Fractional Parts of Cube Roots

### Problem 3: Padding

### Problem 4: Hashes

### Problem 5: Passwords